In [122]:
import json
import csv
import pickle
import random
import pandas as pd
import numpy as np
import networkx as nx
import pprint

import spacy
from spacy import displacy
from spacy.tokens import Span
from langdetect import detect
from cleantext import clean
import re

In [2]:
nlp = spacy.load("en_core_web_lg")

/Users/laura/miniforge3/envs/test/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_lg' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
with open('tweets_1000.json', 'r') as file:
    all_tweets = json.load(file)

In [4]:
import math

def get_type(refs):
    
    if not isinstance(refs, list):
        return "tweet"
    else:
        return refs[0]["type"]
    
def get_lang(text):
    try:
        language = detect(text)
    except:
        language = None
    return language

In [5]:
df_all_tweets = pd.DataFrame.from_dict(all_tweets)
df_all_tweets = df_all_tweets.transpose().set_index("id")
df_all_tweets["type"] = df_all_tweets.apply(lambda row: get_type(row["referenced_tweets"]), axis=1)
df_all_tweets["lang"] = df_all_tweets.apply(lambda row: get_lang(row["text"]), axis=1)
df_all_tweets

text  \
id                                                                       
1501715377335775235  RT @stillgray: U.S. Under Secretary of State V...   
1501734364664590338  @BHDonk1 Russia took Crimea after the U.S. fun...   
1501715719205097472  RT @anneapplebaum: There is always a military ...   
1501725096087588866  RT @RonFilipkowski: It’s Guilfoyle’s birthday,...   
1501734648711245827  RT @EpicSerbian: Russians pre-empted Donbass a...   
...                                                                ...   
1498075055573676033  RT @ataraxie66: Etrange quand même, on trouve ...   
1498075067812614144  @RadioFreeTom Maybe Putin is going through an ...   
1498074904461205504  @Judmir3 Its fake, I don't know what this is b...   
1498075030160293892  RT @FaroPeriodismo: #URGENTE nueva primicia en...   
1498082471308627971  RT @OccupyDemocrats: BREAKING NEWS: Hacker gro...   

                               author_id                created_at  \
id                                                                   
1501715377335775235            115792183  2022-03-10T00:23:41.000Z   
1501734364664590338  1376636130246860801  2022-03-10T01:39:08.000Z   
1501715719205097472            285005616  2022-03-10T00:25:02.000Z   
1501725096087588866           1948262269  2022-03-10T01:02:18.000Z   
1501734648711245827            257097577  2022-03-10T01:40:15.000Z   
...                                  ...                       ...   
1498075055573676033           1135125324  2022-02-27T23:18:20.000Z   
1498075067812614144           3798288154  2022-02-27T23:18:23.000Z   
1498074904461205504  1370452383889772546  2022-02-27T23:17:44.000Z   
1498075030160293892  1338598368474435588  2022-02-27T23:18:14.000Z   
1498082471308627971            152766660  2022-02-27T23:47:48.000Z   

                                                     referenced_tweets  \
id                                                                       
1501715377335775235  [{'type': 'retweeted', 'id': '1501310103529541...   
1501734364664590338  [{'type': 'replied_to', 'id': '150172868701149...   
1501715719205097472  [{'type': 'retweeted', 'id': '1501700149332090...   
1501725096087588866  [{'type': 'retweeted', 'id': '1501706894905585...   
1501734648711245827  [{'type': 'retweeted', 'id': '1501580337993838...   
...                                                                ...   
1498075055573676033  [{'type': 'retweeted', 'id': '1498071752345276...   
1498075067812614144  [{'type': 'replied_to', 'id': '149804628286630...   
1498074904461205504  [{'type': 'replied_to', 'id': '149805059902325...   
1498075030160293892  [{'type': 'retweeted', 'id': '1497966248382005...   
1498082471308627971  [{'type': 'retweeted', 'id': '1498080651500785...   

                                                   context_annotations  \
id                                                                       
1501715377335775235                                                NaN   
1501734364664590338                                                NaN   
1501715719205097472                                                NaN   
1501725096087588866  [{'domain': {'id': '10', 'name': 'Person', 'de...   
1501734648711245827                                                NaN   
...                                                                ...   
1498075055573676033  [{'domain': {'id': '45', 'name': 'Brand Vertic...   
1498075067812614144  [{'domain': {'id': '10', 'name': 'Person', 'de...   
1498074904461205504                                                NaN   
1498075030160293892                                                NaN   
1498082471308627971                                                NaN   

                    withheld  geo        type lang  
id                                                  
1501715377335775235      NaN  NaN   retweeted   en  
1501734364664590338      NaN  NaN  replied_to   en  
1501715719205097472      NaN  NaN   retweeted   en  
1

In [6]:
df_tweets = df_all_tweets[df_all_tweets["type"] == "tweet"]
df_tweets_en = df_tweets[df_tweets["lang"] == "en"]
df_tweets_en

text  \
id                                                                       
1506759271027036167  As many as 15,000 Russian troops have been kil...   
1506196506595975169  Horrible what happened to @navalny today. We a...   
1506178370748436481  An elite Ukrainian drone unit exploits the cov...   
1499601106271363077  STEELWORKS OTH1 /// DJ JETLAGGED /// PRIME NUM...   
1502466077430800387  Coming from the guy who is trying to rule the ...   
1501359087501692932  WSJ News Exclusive | Saudi, Emirati Leaders De...   
1509087938461990913  Russia's foreign minister has arrived in China...   
1506782851081150464  Detained American basketball player Brittney G...   
1506795268389449732  Marketing department here at #Ripple #XRP #BAY...   
1506782887282098179  “Kirill is here as a warrior of Christ,” said ...   
1498812334822862850  I want to believe sanctions will be enough. Go...   
1499625688462069763  We're stunned, as you are. What are your curre...   
1503182989416611843  I ordered a moscow mule last night and this gu...   
1503166678162886662  While You Were Distracted With #Ukraine, #Gove...   
1503174587869138944  You know, I feel bad for Russian people that a...   
1498549338967986177  Fascinating interview with Fiona Hill on Putin...   
1500217170143457291  The mentality that discriminates innocent peop...   
1509136664056119296  Can Montana expect any Ukrainian refugees?  ht...   
1509136552882024451  🇺🇦Head of the Nikolaev Regional State Administ...   
1499897367352467456  https://t.co/pEoMk1ROqT NATO allies rejected U...   
1502034986358255617  Anastasiades: Turkey is undermining sanctions ...   
1506890486874783744  https://t.co/w7VQs4aFbf Apparently Sweden is n...   
1509676096404570112  "China opposes calls to remove Russia from gro...   
1509597773955571718  Has Putin's invasion hurt the European Left? @...   
1509597697803702274  What an account fron Nadia about her beloved c...   
1499130681561079818  #Ukraine Thank goodness for the #EU, otherwise...   
1504031077542813702  @tedcruz \nYour best contribution would be to ...   
1504050507538735106  ⚡️In Chernihiv, in one of the residential dist...   
1507727214896459779  US Army veteran who has volunteered to fight i...   
1507765361692790791  The 700 million dollar yacht is not #Putin's. ...   
1509676565227278337  Watch: Russian state TV host calls for reinsta...   
1498793758057545739  Apple pauses all product sales in Russia, stop...   
1503093023990767617  The logistics of this “Ukrainian war“ seem to ...   
1499984570942427139  Samsung suspends supplies to Russia due to the...   
1499994538299072512  Oh well, now Twitter is officially blocked in ...   
1498719266434023428  Yahoo News: Americans, Canadians answer Ukrain...   
1507323299034026003  Mariupol is in Russian hands. https://t.co/4HZ...   
1502421676687953923  Well, at least Biden talks to Zelensky encoura...   
1502421450904457219  My senator, Marsha Blackburn, voted 'NO' on ai...   
1507427205139283969  Today, 25th March, at 4.30pm, Russian occupier...   
1501242787315924994  BREAKING\n\nThe US BANS RUSSIAN OIL, GAS AND C...   
1501215537220800514            Would absolutely hate to live in russia   
1501173619946364928  Shared with @AymanM the challenge I had trying...   
1507867250585264133  FCC says Russian anti-virus firm Kaspersky pos...   
1503359505060487168  6366 Sanctions are Imposed on russia, yet Ukra...   
1501762900696117249  To, Joy Reid (MSNBC), Nikole Hannah-Jones (161...   
1509331356702150664     Putin doesn’t have an endgame. All #Bidenomics   
1500161826369286148  A love letter to the business community for st...   
1508236563217408001  Russia-Ukraine War Updates: Day 32 of Putin’s ...   
1501887477413339139  Russian former F1 driver Nikita Mazepin and fa...   
1499355233268420613  LET PRO PEACE HACKERS FREEZE ALL BANK ACCOUNTS...   
1508225701316595717  He just can't/won't stop sucking up to putin, ...   
1503307484693504000  The cowardice of the #EU and #NATO leaders?To ...

In [101]:
def get_np_ents(doc):
    
    relevant_entities = ["EVENT",
                         "GPE",
                         "LOC",
                         "NORP",
                         "ORG",
                         "PERSON"
                        ]
    
    ignore_tokens = ["det"]

    new_ents = []

    for np in doc.noun_chunks:
        #print(np, np.ents, len(np.ents))
        if np.ents and any(ent.label_ in relevant_entities for ent in np.ents):
            
            np_ent = np
            np_ent.label_ = "NP"
            
            if np.root.ent_type_: # keep entity label if the noun at the root of the np is that entity
                np_ent.label_ = np.root.ent_type_
            
            while doc[np_ent.start].dep_ in ignore_tokens: # remove "the", "an", etc
                np_ent.start = np_ent.start+1
                
            new_ents.append(np_ent)

    # in order to be able to nicely visualize the entity-noun-phrases, 
    # I'm replacing spacy's predicted named entities with them.
    # This is just because of displacy weirdness though -- they should instead
    # be saved separately as spans.
    doc.ents = new_ents  

    #displacy.render(doc, style="ent")
    
    return(doc)

In [102]:
get_np_ents(nlp(tweets[2]))

An elite Ukrainian drone unit exploits the cover of night to destroy Russian tanks and trucks while their soldiers sleep, report says 

In [22]:
for label in nlp.get_pipe("ner").labels:
    print(label, spacy.explain(label))

CARDINAL Numerals that do not fall under another type
DATE Absolute or relative dates or periods
EVENT Named hurricanes, battles, wars, sports events, etc.
FAC Buildings, airports, highways, bridges, etc.
GPE Countries, cities, states
LANGUAGE Any named language
LAW Named documents made into laws.
LOC Non-GPE locations, mountain ranges, bodies of water
MONEY Monetary values, including unit
NORP Nationalities or religious or political groups
ORDINAL "first", "second", etc.
ORG Companies, agencies, institutions, etc.
PERCENT Percentage, including "%"
PERSON People, including fictional
PRODUCT Objects, vehicles, foods, etc. (not services)
QUANTITY Measurements, as of weight or distance
TIME Times smaller than a day
WORK_OF_ART Titles of books, songs, etc.


In [99]:
def preprocess(tweet):
    tweet = tweet.replace("#", "")
    tweet = tweet.replace(" | ", ". ")
    tweet = tweet.replace("’g", "ing")
    tweet = clean(tweet, lower = False)
    tweet = re.sub(r'http\S+', '', tweet)
    #print(tweet)
    
    doc = nlp(tweet)
    doc = get_np_ents(doc)
    return(doc)

In [107]:
tweets = list(df_tweets_en["text"])
print(len(tweets))

58


In [108]:
tweets = [preprocess(tweet) for tweet in tweets]

In [110]:
entities = []

for tweet in tweets:
    
    print("------------------------------------------")
    
    displacy.render(tweet, style="ent")
    #displacy.render(twtee)
    
    for ent in tweet.ents:
        for token in ent:
            print(token, token.pos_, token.dep_, token.ent_type_)
        if not ent in entities:
            entities.append(ent)
        print()

------------------------------------------


As ADV advmod NP
many ADJ amod NP
as ADP quantmod NP
15,000 NUM nummod NP
Russian ADJ amod NP
troops NOUN nsubjpass NP

Ukraine PROPN pobj GPE

NATO PROPN nsubj ORG

Carlson PROPN pobj PERSON

------------------------------------------


Putin PROPN nsubj PERSON

------------------------------------------


elite ADJ amod NP
Ukrainian ADJ amod NP
drone NOUN compound NP
unit NOUN nsubj NP

Russian ADJ amod NP
tanks NOUN dobj NP

------------------------------------------


STEELWORKS PROPN compound NP
OTH1 PROPN ROOT NP

DJ PROPN compound ORG
JETLAGGED PROPN appos ORG

Red PROPN compound NP
Cross PROPN poss NP
' PART case NP
Ukraine PROPN compound NP
Crisis PROPN compound NP
Appeal PROPN pobj NP

------------------------------------------


...

------------------------------------------


Ukraine PROPN nmod NP
/ SYM punct NP
Russia PROPN nsubj NP

America PROPN pobj GPE

------------------------------------------


Ukrainian ADJ amod WORK_OF_ART
Photojournalist PROPN poss WORK_OF_ART
's PART case WORK_OF_ART
War PROPN compound WORK_OF_ART
Diary PROPN ROOT WORK_OF_ART

------------------------------------------


Russia PROPN nsubj GPE

Ukraine PROPN pobj GPE

Afghanistan PROPN nsubj GPE

------------------------------------------


Russia PROPN pobj GPE



In [111]:
print(len(entities))

138


In [170]:
ent_am = np.zeros([len(entities), len(entities)])

for tweet in tweets:
    
    if len(tweet.ents)>1:
        for ent1 in tweet.ents:
            for ent2 in tweet.ents:
                if not ent1 == ent2:
                    ent_am[entities.index(ent1)][entities.index(ent2)] += 1
    
pprint.pprint(ent_am)

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])


In [171]:
G = nx.from_numpy_matrix(ent_am)
mapping = {i:entities[i].text for i in range(len(entities))}
G = nx.relabel_nodes(G, mapping)

In [172]:
G.degree()

DegreeView({'As many as 15,000 Russian troops': 3, 'Ukraine': 22, 'NATO': 3, 'Carlson': 3, 'Putin': 8, 'elite Ukrainian drone unit': 1, 'Russian tanks': 1, 'STEELWORKS OTH1': 2, 'DJ JETLAGGED': 2, "Red Cross' Ukraine Crisis Appeal": 2, 'WSJ News Exclusive': 3, 'Emirati Leaders': 3, 'Biden': 3, 'Ukraine Crisis': 3, "Russia's foreign minister": 1, 'China': 6, 'Detained American basketball player Brittney Griner': 3, 'US Embassy official': 3, 'Russia': 23, 'State Department Spokesman Ned Price': 3, 'XRPLcommunity ukrainewar defi': 0, 'Russian priest': 3, 'Ukrainian Nazis': 3, 'American multinational corporations': 3, "Putin's first rodeo": 0, 'Russia Ukraine': 0, 'Moscow': 1, 'Russian people': 3, 'not all Russians': 3, 'Fiona Hill': 1, 'NATO Ukrania Rusia UnitedNations Europe Kyiv': 0, 'Montana': 1, 'Ukrainian refugees': 1, 'Nikolaev Regional State Administration Kim': 2, 'UkraineWar Russia': 2, ' NATO allies': 3, "Ukraine's demand": 3, 'broader, even more brutal European war': 3, "Russia

In [165]:
nx.write_gexf(G, "network1.gexf")